# Bacterial Classification

## Goggle Drive

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False
    
%cd drive/My Drive/projects/GDL_code

## imports

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from utils.loaders import load_bacteria

## configuration

In [ ]:
# Generation resolution - Must be square 
# Training data is also scaled to this.
GENERATE_SQUARE = 250
IMAGE_CHANNELS = 3

# Configuration
DATA_PATH = '/content/drive/My Drive/projects/GDL_code/data/bacteria'
EPOCHS = 50
BATCH_SIZE = 32

print(f"Will generate {GENERATE_SQUARE}px square images.")

# run params
SECTION = 'cnn'
RUN_ID = '0001'
DATA_NAME = 'bacteria'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

## load & preprocess images

In [ ]:
# Depending on size of image dataset, initial preprocessing can take a while.
# Because of this time needed, save a Numpy preprocessed file.
# In case this file is large enough to cause problems for some verisons of Pickle,
# we use Numpy binary files instead.
training_data = load_beauty(DATA_PATH, GENERATE_SQUARE, GENERATE_SQUARE, BATCH_SIZE, BUFFER_SIZE, IMAGE_CHANNELS)